In [ ]:
!pip install flask twilio pyngrok
from flask import Flask, request
from twilio.twiml.messaging_response import MessagingResponse
from pyngrok import ngrok
import os

In [ ]:
! pip install -qU groq weasyprint markdown

In [ ]:
from google.colab import userdata
from groq import Groq
import json
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

class SmartEMICalculator:
    def __init__(self, language="English", monthly_salary=0, monthly_expenses=0):
        self.loan_amount = 0
        self.interest_rate = 0
        self.loan_term = 0
        self.monthly_interest_rate = 0
        self.language = language
        self.monthly_salary = monthly_salary
        self.monthly_expenses = monthly_expenses

        try:
            self.api_key = "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"
            self.client = Groq(api_key=self.api_key)
        except:
            raise Exception("Please set GROQ_API_KEY in Colab Secrets")

    def set_loan_details(self, amount, rate, term_years):
        """Set the basic loan details"""
        self.loan_amount = amount
        self.interest_rate = rate
        self.loan_term = term_years * 12
        self.monthly_interest_rate = rate / (12 * 100)

    def set_user_details(self, monthly_salary, monthly_expenses):
        """Set user's financial details"""
        self.monthly_salary = monthly_salary
        self.monthly_expenses = monthly_expenses

    def calculate_emi(self):
        """Calculate the monthly EMI"""
        if self.loan_amount <= 0 or self.interest_rate <= 0 or self.loan_term <= 0:
            return 0
        numerator = self.loan_amount * self.monthly_interest_rate * (1 + self.monthly_interest_rate) ** self.loan_term
        denominator = (1 + self.monthly_interest_rate) ** self.loan_term - 1
        return round(numerator / denominator, 2)

    def calculate_total_interest(self):
        """Calculate total interest paid over the loan term"""
        emi = self.calculate_emi()
        total_amount = emi * self.loan_term
        return round(total_amount - self.loan_amount, 2)

    def calculate_smart_payment_impact(self, extra_payment):
        """Calculate how extra payments reduce loan term and interest"""
        original_emi = self.calculate_emi()
        original_total_interest = self.calculate_total_interest()

        balance = self.loan_amount
        months = 0
        total_interest = 0

        while balance > 0 and months < self.loan_term:
            interest = balance * self.monthly_interest_rate
            principal = original_emi - interest + extra_payment
            balance -= principal
            total_interest += interest
            months += 1

        return {
            'Original_Term': self.loan_term,
            'New_Term': months,
            'Original_Interest': original_total_interest,
            'New_Interest': round(total_interest, 2),
            'Interest_Saved': round(original_total_interest - total_interest, 2)
        }

    def get_amortization_schedule(self, extra_payment=0):
        """Generate amortization schedule for a given extra payment"""
        original_emi = self.calculate_emi()
        balance = self.loan_amount
        schedule = []
        months = 0

        while balance > 0 and months < self.loan_term:
            interest = balance * self.monthly_interest_rate
            principal = original_emi - interest + extra_payment
            balance -= principal
            schedule.append(max(balance, 0))
            months += 1

        return schedule

    def get_groq_recommendations(self, loan_details):
        """Get smart payment recommendations from Groq in specified language"""
        disposable_income = self.monthly_salary - self.monthly_expenses
        try:
            prompt = f"""
            As a financial advisor, give smart payment advice for this loan in {self.language}:

            - Amount: ₹{loan_details['amount']:,}
            - Interest Rate: {loan_details['rate']}%
            - Term: {loan_details['term']} years
            - Monthly EMI: ₹{loan_details['emi']:,}
            - Total Interest: ₹{loan_details['total_interest']:,}
            - User’s Monthly Salary: ₹{self.monthly_salary:,}
            - User’s Monthly Expenses: ₹{self.monthly_expenses:,}
            - Disposable Income: ₹{disposable_income:,}

            Include:
            1. Payment strategies based on user’s income
            2. Effects of extra payments with examples
            3. Ways to shorten the loan term
            4. Tax benefits and other options
            5. Useful tips for Indian borrowers
            """

            response = self.client.chat.completions.create(
                model="qwen-qwq-32b",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.6,
                max_completion_tokens=4096,
                top_p=0.95,
                stream=True,
            )

            recommendations = []
            for chunk in response:
                if chunk.choices:
                    recommendations.append(chunk.choices[0].delta.content or "")

            if not recommendations:
                suggested_extra = min(max(disposable_income - loan_details['emi'], 0), 5000)
                return f"""
### Recommendations for ₹{loan_details['amount']:,} Loan ({loan_details['rate']}% for {loan_details['term']} years)

**Your Finances:**
- Monthly Salary: ₹{self.monthly_salary:,}
- Monthly Expenses: ₹{self.monthly_expenses:,}
- Disposable Income: ₹{disposable_income:,}

1. **Payment Strategies Based on Your Income**
   - Pay the EMI of ₹{loan_details['emi']:,} on time using auto-debit.
   - With ₹{disposable_income:,} left after expenses, add ₹{suggested_extra:,} monthly to your EMI to reduce interest.
   - Use bonuses or extra income (e.g., 10% of your salary) for lump-sum prepayments.

2. **Effects of Extra Payments**
   - **₹{suggested_extra:,} Extra Monthly**: New payment becomes ₹{loan_details['emi'] + suggested_extra:,}. Cuts term from {int(loan_details['term'] * 12)} months to ~{self.calculate_smart_payment_impact(suggested_extra)['New_Term']} months, saving ~₹{self.calculate_smart_payment_impact(suggested_extra)['Interest_Saved']:,}.
   - **₹50,000 Yearly**: Paying this annually reduces term to ~17 years, saving ~₹150,000–₹200,000 in interest.

3. **Ways to Shorten the Loan Term**
   - Increase EMI to ₹{round(loan_details['emi'] * 1.15):,} (15% more) if your salary grows, aiming for a 15-year term.
   - Pay ₹{suggested_extra:,} extra monthly now, then raise it as income increases—could finish in 10–12 years.
   - Use a one-time ₹1 lakh payment (e.g., from savings) to cut ~2–3 years.

4. **Tax Benefits and Other Options**
   - Claim ₹1.5 lakh on principal (Section 80C) and ₹2 lakh on interest (Section 24) yearly if it’s a home loan.
   - If mutual funds earn >{loan_details['rate']}%, invest some of your ₹{disposable_income:,} disposable income there.
   - PPF at 7.1% is less than {loan_details['rate']}%, so prepaying saves more than investing there.

5. **Useful Tips for Indian Borrowers**
   - Confirm no prepayment penalties with your bank.
   - Automate EMI payments to avoid late fees.
   - Keep ₹{round(self.monthly_expenses * 3):,} as an emergency fund before extra payments.
   - If rates drop below {loan_details['rate']}%, consider refinancing.
   - Consult an advisor to optimize based on your goals.
"""
            return "".join(recommendations)

        except Exception as e:
            return f"Error getting recommendations: {str(e)}"

    def generate_detailed_report(self, extra_payment=5000):
        """Generate a comprehensive loan analysis report with dynamic visualizations"""
        emi = self.calculate_emi()
        total_interest = self.calculate_total_interest()
        smart_payment_impact = self.calculate_smart_payment_impact(extra_payment)
        loan_details = {
            'amount': self.loan_amount,
            'rate': self.interest_rate,
            'term': self.loan_term/12,
            'emi': emi,
            'total_interest': total_interest
        }
        recommendations = self.get_groq_recommendations(loan_details)

        # Create visualizations and capture as base64 images
        disposable_income = self.monthly_salary - self.monthly_expenses
        extra_payments = [0, 2500, 5000, min(disposable_income - emi, 7500)]
        extra_payments = [x for x in extra_payments if x >= 0]

        # Visualization 1: Loan Balance Over Time
        plt.figure(figsize=(10, 5))
        months = range(1, self.loan_term + 1)
        colors = ['#FF9999', '#66B2FF', '#99FF99', '#FFCC99']
        for i, extra in enumerate(extra_payments):
            schedule = self.get_amortization_schedule(extra)
            plt.plot(months[:len(schedule)], schedule, label=f'₹{extra:,}/mo', color=colors[i], linewidth=2)
        plt.xlabel('Months', fontsize=10)
        plt.ylabel('Balance (₹)', fontsize=10)
        plt.title('Loan Balance Projection', fontsize=12, pad=15)
        plt.legend(fontsize=8)
        plt.grid(True, linestyle='--', alpha=0.7)
        buf1 = io.BytesIO()
        plt.savefig(buf1, format='png', dpi=300)
        plt.close()
        loan_balance_img = base64.b64encode(buf1.getvalue()).decode('utf-8')

        # Visualization 2: Term Comparison
        plt.figure(figsize=(10, 5))
        categories = [f'₹{extra:,}' for extra in extra_payments]
        terms = [self.calculate_smart_payment_impact(extra)['New_Term']/12 for extra in extra_payments]
        sns.barplot(x=categories, y=terms, palette='Blues')
        plt.ylabel('Years', fontsize=10)
        plt.title('Loan Term Comparison', fontsize=12, pad=15)
        plt.xticks(fontsize=8)
        plt.yticks(fontsize=8)
        buf2 = io.BytesIO()
        plt.savefig(buf2, format='png', dpi=300)
        plt.close()
        term_comparison_img = base64.b64encode(buf2.getvalue()).decode('utf-8')

        # Create structured markdown report
        report = f"""


# Loan Analysis Report

---

### Loan Overview
#### Basic Details
| Parameter          | Value              |
|---------------------|--------------------|
| Loan Amount         | ₹{self.loan_amount:,}  |
| Interest Rate       | {self.interest_rate}%  |
| Loan Term           | {self.loan_term//12} years |
| Monthly EMI         | ₹{emi:,.2f}        |
| Total Interest      | ₹{total_interest:,.2f}  |

#### User Financials
| Category            | Amount             |
|---------------------|--------------------|
| Monthly Salary      | ₹{self.monthly_salary:,} |
| Monthly Expenses    | ₹{self.monthly_expenses:,} |
| Disposable Income   | ₹{disposable_income:,} |

---

### Visual Analysis
![Loan Balance Projection](data:image/png;base64,{loan_balance_img})
![Term Comparison](data:image/png;base64,{term_comparison_img})

---

### Payment Scenarios
#### Smart Payment Impact
| Scenario            | Value              |
|---------------------|--------------------|
| Original Term       | {smart_payment_impact['Original_Term']} months |
| New Term            | {smart_payment_impact['New_Term']} months |
| Interest Saved      | ₹{smart_payment_impact['Interest_Saved']:,.2f} |

#### Payment Strategy
| Component           | Amount             |
|---------------------|--------------------|
| Regular EMI         | ₹{emi:,.2f}        |
| Extra Payment       | ₹{extra_payment:,} |
| Total Payment       | ₹{emi + extra_payment:,.2f} |
| Term Reduction      | {smart_payment_impact['Original_Term'] - smart_payment_impact['New_Term']} months |

---

### AI Recommendations
{recommendations}
        """
        return report


In [ ]:
import pandas as pd
import io
import base64
import markdown
from weasyprint import HTML, CSS

from weasyprint import HTML, CSS

def generate_pdf_report(html_content, logo_path, output_path):
    # Read logo and convert to base64
    with open(logo_path, "rb") as logo_file:
        logo_data = base64.b64encode(logo_file.read()).decode('utf-8')

    css = CSS(string=f'''
        @import url('https://fonts.googleapis.com/css2?family=Noto+Sans:wght@400;700&display=swap');

        /* Define page layout */
        @page {{
            size: A4;
            margin: 1cm;

            /* Header on every page */
            @top-left {{
                content: element(pageHeader);
            }}
        }}

        /* Header styling */
        #divHeader {{
            position: running(pageHeader); /* Define header as reusable element */
            height: 2.5cm;
            width: 100%;
            display: flex;
            align-items: center;
        }}

        /* Body styling */
        body {{
            font-family: 'Noto Sans', sans-serif;
            font-size: 9pt;
            line-height: 1.4;
            margin-top: 3cm; /* Add margin to push content below header */
        }}

        /* Header content styling */
        .header {{
            display: flex;
            align-items: center;
        }}

        .header img {{
            height: 30pt;
            width: 30pt;
            margin-right: 10pt;
        }}

        .header-text {{
            font-size: 14pt;
            color: white;
            font-weight: bold;
        }}

        /* Table styling */
        h1 {{ font-size: 14pt; }}
        h2 {{ font-size: 12pt; }}
        h3 {{ font-size: 11pt; }}
        table {{ width: 100%; border-collapse: collapse; }}
        th, td {{ border: 1px solid #ddd; padding: 4pt; font-size: 8.5pt; }}
        img {{ width: 100%; margin: 10pt 0; }}

        /* Styling for <think> tag */
        think {{
            display: block;
            font-size: 8pt; /* Smaller text */
            border: 1px solid #ccc; /* Border around the content */
            padding: 10pt; /* Padding inside the box */
            margin: 10pt 0; /* Space above and below the box */
            background-color: #f9f9f9; /* Light background */
            position: relative; /* For positioning the title */
        }}

        /* Add "Reasoning" title above the <think> content */
        think::before {{
            content: "Reasoning";
            display: block;
            font-size: 10pt;
            font-weight: bold;
            margin-bottom: 5pt; /* Space between title and content */
            color: #333; /* Darker text for the title */
        }}
    ''')

    HTML(string=html_content).write_pdf(
        output_path,
        stylesheets=[css],
        presentational_hints=True
    )


def generate_loan_suggestions(csv_path, user_loan_amount, user_loan_term,
                             user_salary, user_expenses, language="English"):
    # 1. Process and analyze CSV data
    loan_data = pd.read_csv(csv_path)
    loan_data = loan_data[['bank_name', 'loan_type', 'interest_rate']]
    loan_data = loan_data.dropna(subset=['interest_rate'])
    loan_data['interest_rate'] = pd.to_numeric(loan_data['interest_rate'], errors='coerce')

    # 2. Find optimal loans (lowest rate per category)
    best_personal = loan_data[loan_data['loan_type'] == 'Personal Loan'].nsmallest(1, 'interest_rate')
    best_education = loan_data[loan_data['loan_type'] == 'Education Loan'].nsmallest(1, 'interest_rate')
    best_home = loan_data[loan_data['loan_type'] == 'Home Loan'].nsmallest(1, 'interest_rate')

    top_loans = pd.concat([best_personal, best_education, best_home])

    # 3. Generate smart suggestions
    suggestions = []
    for _, row in top_loans.iterrows():
        calculator = SmartEMICalculator(language, user_salary, user_expenses)
        calculator.set_loan_details(user_loan_amount, row['interest_rate'], user_loan_term)

        # Generate AI-powered recommendation
        report = calculator.generate_detailed_report(extra_payment=5000)

        suggestions.append({
            'Bank': row['bank_name'],
            'Loan Type': row['loan_type'],
            'Interest Rate': row['interest_rate'],
            'Recommendation': report.split('AI Recommendations')[1].split('Smart Payment Strategy')[0].strip()
        })

    # 4. Create comparison summary
    summary = f"""
    === Smart Loan Suggestions ===
    Based on {len(loan_data)} loan products analyzed:

    1. Best Personal Loan: {best_personal['bank_name'].values[0]} ({best_personal['interest_rate'].values[0]}%)
    2. Best Education Loan: {best_education['bank_name'].values[0]} ({best_education['interest_rate'].values[0]}%)
    3. Best Home Loan: {best_home['bank_name'].values[0]} ({best_home['interest_rate'].values[0]}%)

    Detailed recommendations:
    """

    for s in suggestions:
        summary += f"\n--- {s['Bank']} ({s['Loan Type']}) ---\n{s['Recommendation']}\n"

    return summary

# Example usage remains same
calculator = SmartEMICalculator(language="English", monthly_salary=50000, monthly_expenses=30000)
calculator.set_loan_details(1000000, 8.5, 20)
report_content = calculator.generate_detailed_report(5000)

# # Convert to HTML and generate PDF
# html_content = markdown.markdown(report_content, extensions=['tables'])
# generate_pdf_report(html_content, '/content/loansaathi.png', 'Loan_Analysis_Report.pdf')

<ipython-input-12-e1e5e369f877>:210: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=categories, y=terms, palette='Blues')


In [ ]:
from flask import Flask, request
from twilio.twiml.messaging_response import MessagingResponse
from pyngrok import ngrok
import os
import logging

# Install dependencies
!pip install mistralai

from mistralai import Mistral

# Set up logging
logging.basicConfig(level=logging.DEBUG, format="%(asctime)s - %(levelname)s - %(message)s")

# Ngrok Authentication
!ngrok authtoken 2uLFxFk7npOY4bRFjwE82Ob1QJs_4KLdm2SPbJugAzAotnDM
os.environ["MISTRAL_API_KEY"] = "o6HjkKCNRvdHusQgcu5LdusNeCTchuD4"
api_key = os.environ["MISTRAL_API_KEY"]

# Initialize Mistral client
client = Mistral(api_key=api_key)
model = "mistral-large-latest"

app = Flask(__name__)

# Dictionary to store user responses
user_data = {}

# Required fields for loan details
required_fields = ["Loan Amount", "Interest Rate", "Loan Term", "Monthly Salary", "Monthly Expenses"]

@app.route("/bot", methods=["POST"])
def bot():
    global user_data

    incoming_msg = request.values.get("Body", "").strip()
    user_id = request.values.get("From")  # Use phone number as unique identifier
    # Initialize user data if not present
    if user_id not in user_data:
        user_data[user_id] = {}
    print(f"{user_data}")
    missing_fields = [field for field in required_fields if field not in user_data[user_id]]
    print(f"🔍 Missing fields: ",missing_fields)
    print(f"📋 Current user data: ",user_data[user_id])
    print(f"\n📩 Received message from {user_id}: {incoming_msg}")

    response = MessagingResponse()
    final_message = ""  # Ensure variable is always defined
    if len(missing_fields)==1:
        current_field = missing_fields[0]
        if incoming_msg.replace(".", "", 1).isdigit():
            user_data[user_id][current_field] = float(incoming_msg)  # Store as float
            missing_fields.remove(current_field)
        print("completed")
        try:
            # Convert all inputs to float
            loan_amount = float(user_data[user_id]["Loan Amount"])
            interest_rate = float(user_data[user_id]["Interest Rate"])
            loan_term = float(user_data[user_id]["Loan Term"])
            monthly_salary = float(user_data[user_id]["Monthly Salary"])
            monthly_expenses = float(user_data[user_id]["Monthly Expenses"])

            final_message = (
                f"📌 Loan Details:\n"
                f"💰 Loan Amount: {loan_amount:.2f}\n"
                f"📊 Interest Rate: {interest_rate:.2f}%\n"
                f"📅 Loan Term: {loan_term:.2f} years\n"
                f"💵 Monthly Salary: {monthly_salary:.2f}\n"
                f"🏠 Monthly Expenses: {monthly_expenses:.2f}\n"
            )
            logging.debug(f"✅ Final message generated:\n{final_message}")
            response.message(final_message)
            print(final_message)
            calculator = SmartEMICalculator(language="English", monthly_salary=monthly_salary, monthly_expenses=monthly_expenses)
            calculator.set_loan_details(loan_amount, interest_rate,int(loan_term))
            report_content = calculator.generate_detailed_report(5000)

            # Convert to HTML and generate PDF
            html_content = markdown.markdown(report_content, extensions=['tables'])
            generate_pdf_report(html_content, '/content/loansaathi.png', 'Loan_Analysis_Report.pdf')
            logging.debug("✅ Loan details successfully sent!")
        except Exception as e:
            print(f"❌ Error: {e}")
            logging.error(f"Error processing user input: {e}")
            response.message("An error occurred while processing your loan details.")
        print("Deleting User Data after Completition")
        # Reset user data
        del user_data[user_id]

    else:
        # Ask for the next missing field
        current_field = missing_fields[0]
        if incoming_msg.replace(".", "", 1).isdigit():
            user_data[user_id][current_field] = float(incoming_msg)  # Store as float
            missing_fields.remove(current_field)

        if missing_fields:
            chat_response = client.chat.complete(
                model=model,
                messages=[{"role": "user", "content": f"Ask user to enter {missing_fields[0]}."}]
            )
            bot_reply = chat_response.choices[0].message.content
            response.message(bot_reply)

            print(f"📝 Asking for: {missing_fields[0]}")
            logging.debug(f"Asking for next missing field: {missing_fields[0]}")
        else:
            response.message(final_message)

    return str(response)


# Start ngrok and Flask server
public_url = ngrok.connect(5000).public_url
print(f"🌐 Your public URL: {public_url}/bot")
logging.info(f"Your public URL: {public_url}/bot")

app.run(port=5000)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🌐 Your public URL: https://e1ff-35-198-195-34.ngrok-free.app/bot
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


{'whatsapp:+919394401247': {}}
🔍 Missing fields:  ['Loan Amount', 'Interest Rate', 'Loan Term', 'Monthly Salary', 'Monthly Expenses']
📋 Current user data:  {}

📩 Received message from whatsapp:+919394401247: Hi


INFO:werkzeug:127.0.0.1 - - [15/Mar/2025 22:15:55] "POST /bot HTTP/1.1" 200 -


📝 Asking for: Loan Amount
{'whatsapp:+919394401247': {}}
🔍 Missing fields:  ['Loan Amount', 'Interest Rate', 'Loan Term', 'Monthly Salary', 'Monthly Expenses']
📋 Current user data:  {}

📩 Received message from whatsapp:+919394401247: 3000000


...

📝 Asking for: Monthly Salary
{'whatsapp:+919394401247': {'Loan Amount': 3000000.0, 'Interest Rate': 5.0, 'Loan Term': 20.0}}
🔍 Missing fields:  ['Monthly Salary', 'Monthly Expenses']
📋 Current user data:  {'Loan Amount': 3000000.0, 'Interest Rate': 5.0, 'Loan Term': 20.0}

📩 Received message from whatsapp:+919394401247: 100000


INFO:werkzeug:127.0.0.1 - - [15/Mar/2025 22:16:41] "POST /bot HTTP/1.1" 200 -


📝 Asking for: Monthly Expenses
{'whatsapp:+919394401247': {'Loan Amount': 3000000.0, 'Interest Rate': 5.0, 'Loan Term': 20.0, 'Monthly Salary': 100000.0}}
🔍 Missing fields:  ['Monthly Expenses']
📋 Current user data:  {'Loan Amount': 3000000.0, 'Interest Rate': 5.0, 'Loan Term': 20.0, 'Monthly Salary': 100000.0}

📩 Received message from whatsapp:+919394401247: 20000
completed
📌 Loan Details:
💰 Loan Amount: 3000000.00
📊 Interest Rate: 5.00%
📅 Loan Term: 20.00 years
💵 Monthly Salary: 100000.00
🏠 Monthly Expenses: 20000.00



<ipython-input-28-e1e5e369f877>:210: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=categories, y=terms, palette='Blues')
DEBUG:fontTools.ttLib.ttFont:Reading 'maxp' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'maxp' table
DEBUG:fontTools.subset.timer:Took 0.002s to load 'maxp'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'maxp'
INFO:fontTools.subset:maxp pruned
DEBUG:fontTools.ttLib.ttFont:Reading 'cmap' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'cmap' table
DEBUG:fontTools.ttLib.ttFont:Reading 'post' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'post' table
DEBUG:fontTools.subset.timer:Took 0.005s to load 'cmap'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'cmap'
INFO:fontTools.subset:cmap pruned
INFO:fontTools.subset:fpgm dropped
INFO:fontTools.subset:prep dropped
INFO:fontTools.subse

Deleting User Data after Completition


In [ ]:
!pkill ngrok


In [ ]:
# Replace with your actual API keys
NGROK_AUTH_TOKEN = "2uLFxFk7npOY4bRFjwE82Ob1QJs_4KLdm2SPbJugAzAotnDM"
MISTRAL_API_KEY = "o6HjkKCNRvdHusQgcu5LdusNeCTchuD4"

os.environ["MISTRAL_API_KEY"] = MISTRAL_API_KEY
from flask import Flask, request
from twilio.twiml.messaging_response import MessagingResponse
import os
import logging
# Install dependencies
!pip install mistralai Cloudinary
import cloudinary
import cloudinary.uploader
import markdown
cloudinary.config(
    cloud_name="ddyaj0xim",
    api_key="481894792447346",
    api_secret="c4nwnY9pVSZC0zUbVVUHwd5CEfI",
)


from mistralai import Mistral
from pyngrok import ngrok

# Set up logging
logging.basicConfig(level=logging.DEBUG, format="%(asctime)s - %(levelname)s - %(message)s")

# Set up logging
logging.basicConfig(level=logging.DEBUG, format="%(asctime)s - %(levelname)s - %(message)s")

# Ngrok Authentication
!ngrok authtoken 2uLFxFk7npOY4bRFjwE82Ob1QJs_4KLdm2SPbJugAzAotnDM

os.environ["MISTRAL_API_KEY"] = MISTRAL_API_KEY
api_key = os.environ["MISTRAL_API_KEY"]

# Initialize Mistral client
client = Mistral(api_key=api_key)
model = "mistral-large-latest"

app = Flask(__name__)

# Store user responses
user_data = {}

# Required fields for loan details
required_fields = ["Loan Amount", "Interest Rate", "Loan Term", "Monthly Salary", "Monthly Expenses"]

@app.route("/bot", methods=["POST"])
def bot():
    try:
        incoming_msg = request.values.get("Body", "").strip()
        user_id = request.values.get("From")  # Unique identifier (WhatsApp number)

        if not incoming_msg:
            return str(MessagingResponse().message("Invalid input. Please enter a valid response."))

        # Initialize user data if not present
        if user_id not in user_data:
            user_data[user_id] = {}

        missing_fields = [field for field in required_fields if field not in user_data[user_id]]

        print(f"📩 Received message from {user_id}: {incoming_msg}")
        print(f"🔍 Missing fields: {missing_fields}")
        print(f"📋 Current user data: {user_data[user_id]}")

        response = MessagingResponse()

        # Validate and store input
        if missing_fields:
            current_field = missing_fields[0]
            if is_valid_number(incoming_msg):
                user_data[user_id][current_field] = float(incoming_msg)
                missing_fields.remove(current_field)

            if not missing_fields:
                return process_loan_details(user_id, response)

            # Ask for next missing field
            next_field = missing_fields[0]
            bot_reply = f"Please enter your {next_field}:"
            response.message(bot_reply)
            print(f"📝 Asking for: {next_field}")

        return str(response)

    except Exception as e:
        logging.error(f"❌ Error: {e}")
        return str(MessagingResponse().message("An error occurred while processing your request."))

def is_valid_number(value):
    """ Check if input is a valid number (float or integer). """
    try:
        float(value)
        return True
    except ValueError:
        return False

def process_loan_details(user_id, response):
    """ Process loan details and generate a response. """
    try:
        loan_amount = user_data[user_id]["Loan Amount"]
        interest_rate = user_data[user_id]["Interest Rate"]
        loan_term = user_data[user_id]["Loan Term"]
        monthly_salary = user_data[user_id]["Monthly Salary"]
        monthly_expenses = user_data[user_id]["Monthly Expenses"]

        final_message = (
            f"📌 Loan Details:\n"
            f"💰 Loan Amount: {loan_amount:.2f}\n"
            f"📊 Interest Rate: {interest_rate:.2f}%\n"
            f"📅 Loan Term: {loan_term:.2f} years\n"
            f"💵 Monthly Salary: {monthly_salary:.2f}\n"
            f"🏠 Monthly Expenses: {monthly_expenses:.2f}\n"
        )

        response.message(final_message)
        print(f"✅ Final message generated:\n{final_message}")
        calculator = SmartEMICalculator(language="Hindi", monthly_salary=monthly_salary, monthly_expenses=monthly_expenses)
        calculator.set_loan_details(loan_amount, interest_rate,int(loan_term))
        report_content = calculator.generate_detailed_report(5000)

        # Convert to HTML and generate PDF
        html_content = markdown.markdown(report_content, extensions=['tables'])
        generate_pdf_report(html_content, '/content/loansaathi.png', 'Loan_Analysis_Report.pdf')
        pdf_filename = "Loan_Analysis_Report.pdf"
        # Upload PDF to Cloudinary
        upload_result = cloudinary.uploader.upload(
            pdf_filename,
            resource_type="raw",
            use_filename=True,
            unique_filename=False,
            overwrite=True
        )
        if 'secure_url' in upload_result:
            pdf_url = upload_result['secure_url']
            print(pdf_url)
            response.message(f"✅ Loan Analysis Report generated successfully!\n\nDownload: {pdf_url}")
        else:
            response.message("❌ Failed to upload report to Cloudinary.")
        from IPython.display import IFrame
        # Display PDF in an embedded frame
        IFrame('Loan_Analysis_Report.pdf', width=600, height=500)

        print("✅ Loan details successfully sent!")

        # Cleanup user data after completion
        del user_data[user_id]
        print("✅ User data cleared after completion.")


    except Exception as e:
        print(f"❌ Error processing loan details: {e}")
        response.message("An error occurred while processing your loan details.")

    return str(response)

# Start ngrok and Flask server
public_url = ngrok.connect(5000).public_url
logging.info(f"🌐 Your public URL: {public_url}/bot")
print(f"public URL: {public_url}/bot")
app.run(port=5000)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
public URL: https://4ca7-34-168-92-106.ngrok-free.app/bot
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
!lsof -i :5000
!kill -9 566
